In [ ]:
import random
from random import randint
import subprocess
import os
import signal
from subprocess import call
import time
import string
import redis
import urllib
import urllib2
random.seed()

In [292]:
def getRandomString(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def getRandomPortNum():
    return randint(30000,40000)

def getUniqueRandomPortNum():
    p = randint(30000,40000)
    while True:
        unique = True
        for s in server_list:
            p2 = int(s.split(':')[1])
            if p2 == p:
                unique = False
                break
        if unique: return p
        p = randint(30000,40000)
    


def startProc(args):
    return subprocess.Popen(args)

# p should be a process type
def killProc(proc):
    try:
        os.killpg(proc.pid, signal.SIGTERM)
#         call(["kill", "-9",str(proc.pid+1)]) #kill redis server pid+1 works for some reason
#         call(["kill", "-9", str(proc.pid)])
        return True
    except:
        print "Failed to kill proc pid:", proc.pid, " maybe doesn't exist"

def stopServer(addr,flag=True):
    p = addr.split(':')[1]
    s = "redis-cli -p " + str(p) + " shutdown nosave"
    call(s.split())
#     stopMonitoring(addr)
    if flag: server_list.pop(addr, None)
    killDtServer(addr)

def killDtServer(addr):
    p = int(addr.split(":")[1]) + 20000
    c = "lsof -t -i:"+str(p)
    task = subprocess.Popen(c.split(),stdout=subprocess.PIPE)
    pid = task.stdout.read().strip()
    if pid == "":
        print "No such proc:",addr
        return
    print "Killing DT server:", addr, "PID: ", pid
    call(["kill", pid])

def killAllServers():
    for s in server_list:
        print "Killing server:",s," pid:", server_list[s].pid
        #killProc(server_list[s])
        stopServer(s, False)
        
def startServer(ip = "127.0.0.1", port=None):
    if port == None:port=getRandomPortNum()
        
    s = ("redis-server --port " + str(port) + " --appendonly yes --appendfilename appendonly-" + str(port) + ".aof"
    " --dbfilename dump-" + str(port) + ".rdb --logfile " + str(port) + ".log --daemonize yes --loglevel verbose")
#     print s
#     print s.split()
    p = startProc(s.split())
    sp = ip + ":" + str(port)
    print "Started server:",sp, " proc", p.pid
    server_list[sp] = p
    startMonitoring(sp)
    startDtServer(ip + ":" +str(port + 20000))
    time.sleep(1)

def startDtServer(addr):
    print "Starting DT_Server", addr
    os.system("python dt_server.py "+addr.split(":")[0]+" "+ addr.split(":")[1]+ " > pdt_server_" + addr.split(":")[1] + ".txt 2>&1 &")

def startServers(numServers):
    server_list.clear()
    for i in range(numServers):
        startServer(port=getUniqueRandomPortNum())

        
def startMonitoring(addr):
    print "masterP_" + addr
    REDIS_SENTINEL.sentinel_monitor("masterP_" + addr, addr.split(':')[0], addr.split(':')[1], 2)
def stopMonitoring(addr):
    REDIS_SENTINEL.sentinel_remove("masterP_" + addr)

def stopMonitoringAll():
    for i in server_list.keys(): 
        try:
            stopMonitoring(i)
        except:
            i = 0

def startDtSentinel():
    s = "python dt-sentinel.py"

#Broken
def cleanUp():
    print os.listdir('.')
    call(["rm", "*.log"])
#     call(["rm", "-rf", "*.log"])
#     call(["rm", "-rf", "appendonly*.aof"])
#     call(["rm", "-rf", "dump*.rdb"])
#     call(["rm", "-rf", "nodes*.conf"])
#     call(["rm", "-rf", "*.conf"])

def push_to_server(cmd, args, addr=None):
    if addr == None:
        kk = randint(0,len(server_list.keys()) - 1)
        #print "rand:",kk, len(server_list.keys())
        addr = server_list.keys()[ randint(0,len(server_list.keys()) - 1)]
    data = urllib.urlencode(args)
    try:
        saddr = addr.split(":")[0] + ":" + str(int(addr.split(":")[1]) + 20000)
        url = "http://" + saddr + '/' + cmd + '?'
        response = urllib2.urlopen(url+data, timeout = 2)
        #print "POST Request:" + url+data
        result = response.read()
        #print "Response:" + result
        return result
    except:
        #print "No response"
        return "FAIL"

def sendListAsPost(url, slist):
    url = url + '/update_hash_to_back_map?'
#     print url, slist
    s = ''
    for i in slist: s = s + ',' + i
    s = s[1:]
    data = urllib.urlencode({'addrs' : s})
#     print 'encoded data', data
    try:
        response = urllib2.urlopen(url+data)
    #     print "POST Request:" + url+data
        result = response.read()
        return result
    except:
        return "Oops"

def getMastersList():
    return [REDIS_SENTINEL.sentinel_masters()[m]['ip'] + ':' + str(REDIS_SENTINEL.sentinel_masters()[m]['port']) for m in REDIS_SENTINEL.sentinel_masters() if not REDIS_SENTINEL.sentinel_masters()[m]['is_sdown']]

def sendMasterList():
    time.sleep(2)
    ml = getMastersList()
    print "Master list:",ml
    print "Updating masters map in dt servers"
    for url in ml:
        ip,port = url.split(':')
        sendListAsPost('http://'+ip+':'+str(int(port) + 20000), ml)
    print "ok"

def get_key(key):
    return push_to_server("get",{"key":key})

def set_key(key, val):
    return push_to_server("set",{"key":key, "value":val})

def wrapUp():
    killAllServers()
    stopMonitoringAll()
    server_list.clear()

def wrapUp2():
    killAllServers()
    server_list.clear()


In [276]:
server_list = {}
REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)
# cleanUp()
startServers(6)
sendMasterList()

Started server: 127.0.0.1:33825  proc 8829
Starting DT_Server 127.0.0.1:53825
Started server: 127.0.0.1:33700  proc 8833
Starting DT_Server 127.0.0.1:53700
Started server: 127.0.0.1:37380  proc 8841
Starting DT_Server 127.0.0.1:57380
Started server: 127.0.0.1:37790  proc 8845
Starting DT_Server 127.0.0.1:57790
Started server: 127.0.0.1:34005  proc 8849
Starting DT_Server 127.0.0.1:54005
Started server: 127.0.0.1:35877  proc 8853
Starting DT_Server 127.0.0.1:55877
Master list: ['127.0.0.1:37790', '127.0.0.1:34005', '127.0.0.1:37380', '127.0.0.1:35877', '127.0.0.1:33825', '127.0.0.1:33700']
Updating masters map in dt servers
ok


In [ ]:
sendMasterList()

In [ ]:
print set_key("ww","33")
print get_key("ww")

In [278]:
# def test1(r=10):
r = 200
kl = []
for i in range(r):
    rs = getRandomString()
    kl.append(rs)
    set_key(rs,rs)
#     print set_key(rs,rs)
#     print get_key(rs)
print kl

['FQ7YV3', '1PT07W', 'HA3MA7', 'W4IDR5', 'MTWQ9V', 'YEKE3D', 'RL3VGX', '1JYYXF', 'IN7965', 'B3YE4Z', 'UPKP6G', 'BXML5S', 'UHWZWA', 'U3ABNE', 'FCRJRL', 'BCJYLW', 'A6G41J', 'FV2YOP', 'TDNK32', '4I64TU', 'AD5LOF', 'RPCWQR', 'LL0RSS', 'EURMEE', 'T74M7V', 'H2T4TV', 'BPNDJ1', 'AXR6FV', 'FLXOWT', 'LHM9HC', 'PG84NA', 'T19FYG', 'NKIUV3', '9TRKFM', 'HA1OP9', '5EWSDO', 'Y8JGW8', 'RQKS09', '1KOUAI', '5PRW43', '0E0J9A', 'U4HIAM', '9UZ4B4', '3H6Q16', 'DTQZK3', '8UA2ET', '3WP8RP', 'J0AIUY', 'PX5FIV', '1ZF3OT', 'UCRD2H', '3GECHC', '82MIZ2', 'NZT4LQ', 'DU5FVQ', 'DERE3E', 'HBEUYM', 'C5CLVU', 'PQOO7X', 'FHWQRG', 'LXQEUP', 'JXQSJG', 'VN933C', 'WLBRSQ', 'CMJ6VA', '45ALR6', 'YVYRVK', 'BQFMPC', 'D5VCM6', '8DKD1A', 'N3N3ML', 'QI6WWI', '9DFMDR', 'BITMD4', 'YMISRU', 'ER5LR1', '50EM9R', 'SY24N7', 'IJK1LV', '8ZEV5L', 'J28GX4', 'D6Z4QY', 'L8WF1G', 'IHATY4', '19MHMQ', 'FUTPJ9', '2DZ9YJ', '0BS2VP', '9FB75A', '4DE19H', 'BXGH7W', '25ONSI', '6A1QJZ', 'NBG8Z8', 'TUO31O', '6ZN9I9', 'G1JLDO', 'Y7HR4X', 'EP63GD', '9C00LN',

In [ ]:
stopServer(server_list.keys()[randint(0,len(server_list.keys()) - 1)])

In [ ]:
#get and write the status to some log
def getStatus(address=None):
    if address is None:
        address=server_list.keys()[randint(0,len(server_list.keys()) - 1)]
    try:
        url = '/getKeyDist'
        port = str((int)(address.split(':')[1])+20000)
        req = 'http://'+address.split(':')[0]+':'+port
        print req+url
        response = urllib2.urlopen(req+url)
        return str(response.read())
    except Exception, e:
        print e
        return "Error retieveing status"

def getAllStatus():
    s = getStatus()[1:-1]
    l = s.split(',')
    for i in l: print i
        
#getAllStatus()

In [297]:
getAllStatus()

http://127.0.0.1:58572/getKeyDist
127.0.0.1:37979-11
127.0.0.1:33700-31
127.0.0.1:38572-25
127.0.0.1:34005-33
127.0.0.1:33825-32
127.0.0.1:35877-9
127.0.0.1:34687-9
127.0.0.1:37790-19
127.0.0.1:32276-20
127.0.0.1:37380-11


In [299]:
getAllStatus()

http://127.0.0.1:52276/getKeyDist
127.0.0.1:37979-11
127.0.0.1:33700-31
127.0.0.1:38572-25
127.0.0.1:33825-65
127.0.0.1:35877-9
127.0.0.1:34687-9
127.0.0.1:37790-19
127.0.0.1:32276-20
127.0.0.1:37380-11


In [296]:
startServer()

Started server: 127.0.0.1:32276  proc 9253
masterP_127.0.0.1:32276
Starting DT_Server 127.0.0.1:52276


In [298]:
stopServer('127.0.0.1:34005')

Killing DT server: 127.0.0.1:34005 PID:  8852


In [ ]:
wrapUp2()

In [ ]:
wrapUp()

In [ ]:

    
def getRandomString():
    
    
# def startSentinel():

    
def killSentinel():
    

#start redis + DT server + notify sentinel
#update server_list
    

def startServers(numServers):

    

#kill the redis server at addr
#update server_list
def killServer(addr):
    

#get and write the status to some log
def getStatus():
    
#compute num migrations from two succesie status calls and write to file
def getNumMigrations():